In [1]:
import os
import warnings
from collections import defaultdict

import pandas as pd
import torch
from dotenv import load_dotenv
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from tqdm.auto import tqdm, trange

load_dotenv("../env")

warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [2]:
from luminar.document.data import (
    FeatureDataset,
    PaddingDataloader,
    n_way_split,
)
from luminar.document.model import ConvolutionalLayerSpec, DocumentClassficationModel
from luminar.features import FeatureExtractor, OneDimFeatures, Slicer, TwoDimFeatures
from luminar.mongo import MongoFindDataset

In [3]:
config = {
    "seed": 42,
    "eval_split": 0.1,
    "test_split": 0.2,
    "feature_model": "meta-llama/Llama-3.2-1B",
    # "feature_model": "gpt2",
}

In [4]:
agents = [
    "human",
    "claude",
    "gpt",
    "gpt_prompt1",
    "gpt_prompt2",
    "gpt_semantic",
    "gpt_writing",
]

In [5]:
# feature_dim = OneDimFeatures(128)
# featurizer = FeatureExtractor.Likelihood()
# config["second_dim_as_channels"] = False
feature_dim = TwoDimFeatures(256, 13)
featurizer = FeatureExtractor.IntermediateLogits(13)
config["second_dim_as_channels"] = True

# slicer = Slicer.First(feature_dim[0])
# slicer = Slicer.Random(feature_dim[0])
slicer = Slicer.RandomMultiple(feature_dim[0] // 4, multiple=4, stride=16)
# slicer = Slicer.RandomMultiple(feature_dim[0] // 4, 4)

config["feature_dim"] = feature_dim
config["featurizer"] = repr(featurizer)
config["slicer"] = repr(slicer)

config["num_samples"] = None


def featurize(dataset) -> FeatureDataset:
    return FeatureDataset(
        tqdm(dataset, desc="Featurizing", leave=False),
        slicer,
        featurizer,
        num_samples=config["num_samples"],
        label_field="label",
        label_zero="human",
    )

In [14]:
config["seed"] = 42

sizes = [0.1] * 10
ai_splits = {}
for agent in tqdm(["human", "gpt"], desc="Domains", position=0):
# for agent in tqdm(agents, desc="Domains", position=0):
    seed_everything(config["seed"])
    ai_splits[agent] = [
        featurize(subset)
        for subset in n_way_split(
            MongoFindDataset(
                {"document.agent": agent, "model.name": config["feature_model"]},
                projection={
                    "_id": 1,
                    "features": [
                        {
                            "label": "$document.label",
                            "agent": "$document.agent",
                            "type": "$document.type",
                            "split": "$split",
                            "transition_scores": "$transition_scores",
                        }
                    ],
                },
                mongo_db_connection=os.environ.get("MONGO_DB_CONNECTION"),
                collection="features_Ghostbuster",
                update_cache=True,
            ).load(),
            *sizes,
        )
    ]

Domains:   0%|          | 0/2 [00:00<?, ?it/s]

Seed set to 42


[MongoFindDataset] Loading Documents from MongoDB: 0it [00:00, ?it/s]

[MongoFindDataset] Writing Cache File /nvme/.cache/luminar/ca54ea93ce3cca72675467985a01c63c2c8f884e8f994cf76d34bcafc9ccd2c8.pkl


Featurizing:   0%|          | 0/13 [00:00<?, ?it/s]

...

Featurizing:   0%|          | 0/12 [00:00<?, ?it/s]

Featurizing:   0%|          | 0/12 [00:00<?, ?it/s]

Seed set to 42


[MongoFindDataset] Loading Documents from MongoDB: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [7]:
human_subsets = ai_splits.pop("human")

## In-Domain Training & Evaluation

In [8]:
from torch.utils.data import ConcatDataset

config |= {
    "batch_size": 32,
}


# capturing config from "closure"
def get_dataloader(*dataset, **kwargs) -> PaddingDataloader:
    if len(dataset) == 1:
        dataset = dataset[0]
    else:
        dataset = ConcatDataset(dataset)
    return PaddingDataloader(
        dataset,
        feature_dim=config["feature_dim"],
        batch_size=config["batch_size"],
        **kwargs,
    )

In [9]:
config |= {
    "projection_dim": 32,
    "learning_rate": 0.0001,
    "warmup_steps": 80,
    "max_epochs": 50,
    "gradient_clip_val": 1.0,
}

# SeqXGPT Layer Configuration
config["conv_layer_shapes"] = [
    ConvolutionalLayerSpec(64, 5),
    *[ConvolutionalLayerSpec(128, 3)],
    ConvolutionalLayerSpec(64, 3),
]

In [10]:
ai_subsets = ai_splits[agent]
seed_everything(config["seed"])

# human_subsets.insert(0, human_subsets.pop())
# ai_subsets.insert(0, ai_subsets.pop())

eval_dataloader = get_dataloader(human_subsets[0], ai_subsets[0])
test_dataloader = get_dataloader(*human_subsets[1:3], *ai_subsets[1:3])
train_dataloader = get_dataloader(*human_subsets[3:], *ai_subsets[3:], shuffle=True)

model = DocumentClassficationModel(**config)
trainer = Trainer(
    max_epochs=config["max_epochs"],
    logger=pl_loggers.TensorBoardLogger(
        save_dir=f"logs/Ghostbuster/in_domain/{type(featurizer).__name__}",
        name=agent,
    ),
    gradient_clip_val=config["gradient_clip_val"],
    callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=10)],
    deterministic=True,
)
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=eval_dataloader,
)
(metrics,) = trainer.test(model, test_dataloader, verbose=False)
pd.DataFrame([metrics])

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

,test_loss,test_f1@0.5,test_acc@0.5,test_f1@best,test_acc@best,test_roc_auc
0,0.047894,0.991326,0.983568,0.992556,0.985915,0.995192


In [11]:

results_in_domain = defaultdict(list)
for agent, ai_subsets in tqdm(ai_splits.items()):
    for _ in trange(5, desc=agent, position=1):
        seed_everything(config["seed"])

        human_subsets.insert(0, human_subsets.pop())
        ai_subsets.insert(0, ai_subsets.pop())
        eval_dataloader = get_dataloader(human_subsets[0], ai_subsets[0])
        test_dataloader = get_dataloader(*human_subsets[1:3], *ai_subsets[1:3])
        train_dataloader = get_dataloader(
            *human_subsets[3:], *ai_subsets[3:], shuffle=True
        )

        model = DocumentClassficationModel(**config)
        trainer = Trainer(
            max_epochs=config["max_epochs"],
            logger=pl_loggers.TensorBoardLogger(
                save_dir=f"logs/Ghostbuster/in_domain/{type(featurizer).__name__}",
                name=agent,
            ),
            gradient_clip_val=config["gradient_clip_val"],
            callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
            deterministic=True,
        )
        trainer.progress_bar_callback.disable()

        trainer.fit(
            model,
            train_dataloaders=train_dataloader,
            val_dataloaders=eval_dataloader,
        )
        (metrics,) = trainer.test(model, test_dataloader, verbose=False)
        results_in_domain[agent].append(metrics)

        # metrics = []
        # for other, subsets in splits.items():
        #     trainer.validate(model, get_dataloader(subsets[-1]), verbose=False)
        #     metrics.append(
        #         {"other": other}
        #         | trainer.test(model, get_dataloader(*subsets[:2]), verbose=False)[0]
        #     )
        # results_in_domain[agent].append(metrics)

        print(agent, metrics)


  0%|          | 0/1 [00:00<?, ?it/s]

gpt:   0%|          | 0/5 [00:00<?, ?it/s]

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | tra

gpt {'test_loss': 0.021219683811068535, 'test_f1@0.5': 0.9962546825408936, 'test_acc@0.5': 0.9929577708244324, 'test_f1@best': 0.9925373196601868, 'test_acc@best': 0.98591548204422, 'test_roc_auc': 0.9987499713897705}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?         

...

gpt {'test_loss': 0.03332681581377983, 'test_f1@0.5': 0.9950248599052429, 'test_acc@0.5': 0.9905660152435303, 'test_f1@best': 0.9962639808654785, 'test_acc@best': 0.9929245114326477, 'test_roc_auc': 0.9986458420753479}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?         

gpt {'test_loss': 0.03907044231891632, 'test_f1@0.5': 0.9950124621391296, 'test_acc@0.5': 0.9905882477760315, 'test_f1@best': 0.9950124621391296, 'test_acc@best': 0.9905882477760315, 'test_roc_auc': 0.9929999709129333}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?         

gpt {'test_loss': 0.05252371355891228, 'test_f1@0.5': 0.9900249242782593, 'test_acc@0.5': 0.9812206625938416, 'test_f1@best': 0.988875150680542, 'test_acc@best': 0.9788732528686523, 'test_roc_auc': 0.9915384650230408}


In [12]:
import pandas as pd

df = pd.DataFrame(
    [
        {
            "agent": agent,
            **metric,
        }
        for agent, metrics in results_in_domain.items()
        for metric in metrics
    ]
)
df = df.groupby("agent").mean()
print(df.to_latex(float_format="\\np{%.3f}"))
df

\begin{tabular}{lrrrrrr}
\toprule
 & test_loss & test_f1@0.5 & test_acc@0.5 & test_f1@best & test_acc@best & test_roc_auc \\
agent &  &  &  &  &  &  \\
\midrule
gpt & \np{0.035} & \np{0.995} & \np{0.990} & \np{0.993} & \np{0.987} & \np{0.996} \\
\bottomrule
\end{tabular}



,test_loss,test_f1@0.5,test_acc@0.5,test_f1@best,test_acc@best,test_roc_auc
agent,,,,,,
gpt,0.034862,0.994514,0.989655,0.993292,0.987307,0.996127


## Cross-Domain Training & Evaluation

In [13]:
results_cross_domain = defaultdict(list)
for _ in trange(5, position=0):
    seed_everything(config["seed"])

    human_subsets.insert(0, human_subsets.pop())
    for ai_subsets in ai_splits.values():
        ai_subsets.insert(0, ai_subsets.pop())

    eval_dataloader = get_dataloader(
        human_subsets[0],
        *[ai_subsets[0] for ai_subsets in ai_splits.values()],
    )
    train_dataloader = get_dataloader(
        *human_subsets[3:],
        *[subset for ai_subsets in ai_splits.values() for subset in ai_subsets[3:]],
        shuffle=True,
    )

    model = DocumentClassficationModel(**config)
    trainer = Trainer(
        max_epochs=config["max_epochs"],
        logger=pl_loggers.TensorBoardLogger(
            save_dir=f"logs/Ghostbuster/cross_domain/{type(featurizer).__name__}",
            name=agent,
        ),
        gradient_clip_val=config["gradient_clip_val"],
        callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
        deterministic=True,
    )
    trainer.progress_bar_callback.disable()

    trainer.fit(
        model,
        train_dataloaders=train_dataloader,
        val_dataloaders=eval_dataloader,
    )

    for agent, ai_subsets in tqdm(ai_splits.items(), position=1):
        (metrics,) = trainer.test(
            model, get_dataloader(*human_subsets[1:3], *ai_subsets[1:3]), verbose=False
        )
        results_cross_domain[agent].append(metrics)
        print(agent, metrics)


  0%|          | 0/5 [00:00<?, ?it/s]

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes
------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 53.6 K | train | ?           | ?        
1 | projection  | Sequential        | 524 K  | train | [32, 16384] | [32, 32] 
2 | classifier  | Linear            | 33     | train | [32, 32]    | [32, 1]  
3 | criterion   | BCEWithLogitsLoss | 0      | tra

  0%|          | 0/1 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Seed set to 42


gpt {'test_loss': 0.06204656511545181, 'test_f1@0.5': 0.9873737096786499, 'test_acc@0.5': 0.9765258431434631, 'test_f1@best': 0.9962639808654785, 'test_acc@best': 0.9929577708244324, 'test_roc_auc': 0.9975000023841858}


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(
    [
        {
            "agent": agent,
            **metric,
        }
        for agent, metrics in results_cross_domain.items()
        for metric in metrics
    ]
)
df = df.groupby("agent").mean()
print(df.to_latex(float_format="\\np{%.3f}"))
df

\begin{tabular}{lrrrrrr}
\toprule
 & test_loss & test_acc@0.5 & test_f1@0.5 & test_acc@best & test_f1@best & test_roc_auc \\
agent &  &  &  &  &  &  \\
\midrule
claude & \np{0.507} & \np{0.817} & \np{0.841} & \np{0.834} & \np{0.854} & \np{0.951} \\
gpt & \np{0.504} & \np{0.819} & \np{0.844} & \np{0.837} & \np{0.857} & \np{0.956} \\
gpt_prompt1 & \np{0.500} & \np{0.820} & \np{0.844} & \np{0.839} & \np{0.859} & \np{0.954} \\
gpt_prompt2 & \np{0.486} & \np{0.825} & \np{0.850} & \np{0.842} & \np{0.862} & \np{0.961} \\
gpt_semantic & \np{0.491} & \np{0.824} & \np{0.849} & \np{0.839} & \np{0.859} & \np{0.960} \\
gpt_writing & \np{0.478} & \np{0.829} & \np{0.854} & \np{0.846} & \np{0.866} & \np{0.968} \\
\bottomrule
\end{tabular}



,test_loss,test_acc@0.5,test_f1@0.5,test_acc@best,test_f1@best,test_roc_auc
agent,,,,,,
claude,0.507301,0.816691,0.841416,0.833952,0.853843,0.950950
gpt,0.503880,0.819192,0.843757,0.836702,0.856563,0.956135
gpt_prompt1,0.500157,0.819693,0.844117,0.838961,0.858809,0.954356
gpt_prompt2,0.486434,0.825205,0.849934,0.842217,0.862103,0.960924
gpt_semantic,0.491409,0.823956,0.848741,0.838713,0.858542,0.959760
gpt_writing,0.477959,0.829464,0.854239,0.845721,0.865608,0.968103


In [ ]:
import pandas as pd


def df_to_latex_heatmap(_df: pd.DataFrame):
    print(
        "\\plotHeatmap{"
        + ",".join(_df.index)
        + "}{%\n    "
        + ",%\n    ".join(
            [
                "{"
                + ",".join(f"{val:.4f}/{round(val, 2):.2f}" for val in row[1:])
                + "}"
                for row in _df.reset_index().values
            ]
        )
        + "%\n}{"
        + ",".join(_df.columns)
        + "}"
    )


In [ ]:
_metric = "test_roc_auc"
# _metric = "test_acc@best"

results = []
for ai_counterpart in sources:
    results.append(
        [
            results_in_domain[ai_counterpart]["metrics"][i][_metric]
            for i in range(len(sources))
        ]
    )
results.append([m[_metric] for m in metrics_cross_domain[: len(sources)]])

df = pd.DataFrame(results, columns=sources, index=list(sources) + ["ALL"])
df["AVG"] = df.mean(axis=1)
df["AVG"][-1] = metrics_cross_domain[-1][_metric]
df

NameError: name 'sources' is not defined

In [ ]:
df_to_latex_heatmap(df)

In [ ]:
config

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

ax = sns.heatmap(
    df,
    annot=True,
    fmt=".2f",
    vmax=1.0,
    vmin=0.0,
    cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True),
    yticklabels=list(sources) + ["ALL"],
    xticklabels=list(sources) + ["AVG"],
    square=True,
    # reduce annotation font size
    annot_kws={"fontsize": 8},
    cbar=False,
)

# rotate x-axis labels by 45 degrees
# anchored at the right edge of the axes
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment("right")

plt.tight_layout()
# plt.savefig(
#     "../figures/evaluation-trained_in_domain-test_0.1-gpt2_256-rand_4-il_13_as_channels.pdf",
#     dpi=300,
# )
plt.show()

### LLR on Whole Datasets

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores
from simple_dataset import Dataset as SimpleDataset
from transition_scores.data import TransitionScores

results_llr = []
for ai_counterpart, split in datasets.items():
    dataset_test = (
        SimpleDataset(split)
        .flat_map(lambda doc: doc["features"])
        .map(
            lambda x: {
                "llr": llr_from_transition_scores(
                    TransitionScores(**x["transition_scores"])
                ),
                "labels": int(x["label"] != "human"),
            },
            in_place=False,
        )
    )
    fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
    auroc = auc(fpr, tpr)

    preds = np.array(dataset_test["llr"])
    labels = np.array(dataset_test["labels"])

    mean_0 = float(np.mean(preds[labels == 0]))
    mean_1 = float(np.mean(preds[labels == 1]))

    thresholds = np.linspace(round(mean_0, 1) - 0.2, round(mean_1, 1) + 0.3, 5001)
    preds_thresholded: np.ndarray = preds > thresholds.reshape(-1, 1)
    acc_thresholded = np.mean((preds_thresholded == labels), axis=1)
    idx = np.argmax(acc_thresholded)
    best_threshold = thresholds[idx]
    best_acc = acc_thresholded[idx]

    results_llr.append(
        {
            "domain": ai_counterpart,
            "auroc": auroc,
            "best_acc": best_acc,
            "best_threshold": best_threshold,
        }
    )

pd.DataFrame(results_llr)

### LLR on Test Splits

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores
from simple_dataset import Dataset as SimpleDataset
from transition_scores.data import TransitionScores

results_llr = []
for ai_counterpart, split in test_splits.items():
    dataset_test = (
        SimpleDataset(split)
        .flat_map(lambda doc: doc["features"])
        .map(
            lambda x: {
                "llr": llr_from_transition_scores(
                    TransitionScores(**x["transition_scores"])
                ),
                "labels": int(x["label"] != "human"),
            },
            in_place=False,
        )
    )
    fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
    auroc = auc(fpr, tpr)

    preds = np.array(dataset_test["llr"])
    labels = np.array(dataset_test["labels"])

    mean_0 = float(np.mean(preds[labels == 0]))
    mean_1 = float(np.mean(preds[labels == 1]))

    thresholds = np.linspace(round(mean_0, 1) - 0.2, round(mean_1, 1) + 0.3, 1001)
    preds_thresholded: np.ndarray = preds > thresholds.reshape(-1, 1)
    acc_thresholded = np.mean((preds_thresholded == labels), axis=1)
    idx = np.argmax(acc_thresholded)
    best_threshold = thresholds[idx]
    best_acc = acc_thresholded[idx]

    results_llr.append(
        {
            "domain": ai_counterpart,
            "auroc": auroc,
            "best_acc": best_acc,
            "best_threshold": best_threshold,
        }
    )

pd.DataFrame(results_llr)

In [ ]:
raise RuntimeError("STOP")

In [ ]:
sample = train_dataset[0]["features"].numpy().squeeze()
train_dataset[0]["labels"]

In [ ]:
fig = plt.imshow(sample.T, cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True))
fig.axes.set_axis_off()
plt.show()

In [ ]:
sample = train_dataset[2]["features"].numpy().squeeze()
train_dataset[2]["labels"]

In [ ]:
fig = plt.imshow(sample.T, cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True))
fig.axes.set_axis_off()
plt.show()

In [ ]:
raise RuntimeError("STOP")

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores

dataset_test = dm._dataset_test.map(
    lambda x: {
        "llr": llr_from_transition_scores(x["features"]),
        "labels": x["labels"],
    },
    in_place=False,
)
fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
auc(fpr, tpr)